# 🛠️ Super-Notebook: **pyenv + Poetry** (Instalación, versiones de Python, entornos, deps y más)

Este notebook es un **ejecutable** para dominar **pyenv** (gestión de múltiples versiones de Python) y **Poetry** (gestión de dependencias, entornos, build y publicación). Incluye **todas las configuraciones típicas**, tips para **WSL/Ubuntu**, ejemplos multi‑proyecto, **scripts**, **export**, **troubleshooting** y más.

> Los bloques de código están listos para copiar/pegar en tu terminal. Puedes adaptar los nombres de proyecto y rutas según tu caso.

## 1) Instalar **pyenv** en Ubuntu/WSL

Instala dependencias de compilación y luego pyenv.

In [ ]:
# Dependencias de compilación (Ubuntu/WSL)
# sudo apt update
# sudo apt install -y build-essential curl git zlib1g-dev libssl-dev \
#   libffi-dev libreadline-dev libsqlite3-dev libbz2-dev libncurses5-dev \
#   liblzma-dev tk-dev ca-certificates

# Instalar pyenv (curl)
# curl https://pyenv.run | bash

# Inicializar pyenv en tu shell (bash/zsh) - agrega a ~/.bashrc o ~/.zshrc:
# export PYENV_ROOT="$HOME/.pyenv"
# command -v pyenv >/dev/null || export PATH="$PYENV_ROOT/bin:$PATH"
# eval "$(pyenv init -)"
# (opcional) eval "$(pyenv virtualenv-init -)"

# Recargar shell
# exec $SHELL -l

# Verificar
# pyenv --version

## 2) Instalar **versiones de Python** con pyenv

Compila e instala múltiples runtimes de Python en tu `$HOME/.pyenv/versions`.

In [ ]:
# Listar versiones disponibles para instalar
# pyenv install -l | less

# Instalar una versión
# pyenv install 3.12.4
# pyenv install 3.13.1

# Ver versiones instaladas
# pyenv versions

# Selección global (para todo el sistema del usuario)
# pyenv global 3.13.1

# Selección local (por proyecto: crea .python-version en el directorio)
# cd ~/proyectos/mi-repo
# pyenv local 3.12.4
# python -V   # debería decir 3.12.4

## 3) Entornos virtuales: `venv` estándar vs `pyenv-virtualenv`

Tienes dos caminos. Ambos funcionan bien con Poetry.

In [ ]:
# A) venv estándar dentro del proyecto
# cd ~/proyectos/mi-repo
# pyenv local 3.12.4
# hash -r
# python -m venv .venv
# source .venv/bin/activate

# B) pyenv-virtualenv (si instalaste el plugin)
# pyenv virtualenv 3.12.4 mi-repo-3124
# pyenv local mi-repo-3124
# pyenv activate mi-repo-3124
# (para salir) pyenv deactivate

## 4) Instalar **Poetry** (recomendado con `pipx`)

Usar `pipx` evita mezclar dependencias globales. Si no tienes `pipx`, instálalo.

In [ ]:
# Instalar pipx (si te falta)
# sudo apt update
# sudo apt install -y pipx
# pipx ensurepath
# exec $SHELL -l

# Instalar Poetry
# pipx install poetry

# Verificar
# poetry --version

## 5) Configuración esencial de **Poetry**

Ajustes útiles para que Poetry cree entornos dentro del proyecto y use tu Python activo.

In [ ]:
# Crear venvs dentro del proyecto (./.venv)
# poetry config virtualenvs.in-project true

# Respetar el Python activo (útil si fijaste con pyenv local)
# poetry config virtualenvs.prefer-active-python true

# (Opcional) Ajustar concurrencia del instalador
# poetry config installer.max-workers 8

# Ver todas tus configs
# poetry config --list

## 6) Inicializar un proyecto con **Poetry**

Puedes crear un proyecto nuevo o inicializar uno existente.

In [ ]:
# Proyecto nuevo (estructura + pyproject.toml)
# poetry new mi_proyecto
# cd mi_proyecto

# Proyecto existente (genera pyproject.toml interactivo)
# cd ~/proyectos/mi-repo
# poetry init

## 7) Fijar versión de Python en `pyproject.toml`

Alinea el runtime (pyenv local) con la restricción de `requires-python`.

```toml
[tool.poetry]
name = "mi-proyecto"
version = "0.1.0"
description = "Demo con pyenv + Poetry"
authors = ["Tu Nombre <tu@email>"]
readme = "README.md"
packages = [{ include = "mi_proyecto", from = "src" }]

[tool.poetry.dependencies]
python = "~3.12" # o ">=3.12,<3.13"

[build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"
```

## 8) Crear/usar el entorno virtual e instalar dependencias

Con la config de la sección 5, Poetry creará `.venv` dentro del proyecto.

In [ ]:
# Crear/usar venv e instalar (lee pyproject + poetry.lock si existe)
# poetry install

# Activar shell del venv
# poetry shell

# O ejecutar un comando dentro del venv sin activar shell
# poetry run python -V

## 9) Gestión de dependencias con **Poetry**

Soporta grupos (`--group`), extras, Git, path, markers y más.

In [ ]:
# Añadir deps de runtime
# poetry add boto3 pandas

# Añadir deps de desarrollo (grupo dev por defecto)
# poetry add --group dev pytest ruff black

# Crear un grupo custom (ej. "lint") y añadir deps
# poetry add --group lint ruff

# Quitar deps
# poetry remove pandas

# Añadir desde Git
# poetry add git+https://github.com/psf/requests.git

# Añadir desde un path local (editable por defecto)
# poetry add ../libs/mi-lib

# Añadir con extras y markers
# poetry add "uvicorn[standard]" --markers "platform_system == 'Linux'"

## 10) Lockfile, actualización y export

Controla versiones exactas en `poetry.lock`, actualiza y exporta para compatibilidad.

In [ ]:
# (Re)generar lock
# poetry lock

# Actualizar deps (respeta restricciones de pyproject)
# poetry update
# poetry update requests

# Exportar a requirements.txt (por ejemplo, para Lambda)
# poetry export -f requirements.txt -o requirements.txt --without-hashes
# (o con hashes si tu entorno lo permite) --with-hashes

## 11) Scripts ejecutables (entry points) con Poetry

Define comandos en `pyproject.toml` y ejecútalos con `poetry run`.

```toml
[tool.poetry.scripts]
mi-comando = "mi_proyecto.cli:main"
```

In [ ]:
# Ejecutar tu script
# poetry run mi-comando

# O directamente un módulo/script
# poetry run python -m mi_proyecto.cli

## 12) Build y publicación en PyPI (o repos privados)

Poetry puede crear `sdist` y `wheel` y publicarlos con token.

In [ ]:
# Construir paquete (dist/)
# poetry build

# Configurar token de PyPI (almacenado en keyring/config)
# poetry config pypi-token.pypi pypi-XXXXXXXXXXXX

# Publicar
# poetry publish --username __token__ --password $POETRY_PYPI_TOKEN_PYPI
# (si configuraste el token con poetry config, puedes omitir credenciales)

## 13) Ejemplos multi-proyecto / multi‑Python

Cada repo puede fijar su Python con `pyenv local` y su `python` en `pyproject.toml`.

In [ ]:
# repo-a (3.12)
# cd ~/proyectos/repo-a
# pyenv install 3.12.4
# pyenv local 3.12.4
# poetry config virtualenvs.in-project true
# poetry config virtualenvs.prefer-active-python true
# poetry init -n
# poetry add boto3 pandas
# poetry install
# poetry run python -V  # 3.12.x

# repo-b (3.13)
# cd ~/proyectos/repo-b
# pyenv install 3.13.1
# pyenv local 3.13.1
# poetry init -n
# poetry add fastapi uvicorn
# poetry install
# poetry run python -V  # 3.13.x

## 14) Configuraciones avanzadas de Poetry

Repositorios alternos, proxies, instalador, y más.

```bash
# Repos privados / alternos
poetry source add my-internal https://pypi.internal.example/simple
poetry source show --tree

# Proxies (si tu red lo requiere)
poetry config http-basic.myproxy USERNAME PASSWORD
# o variables de entorno: HTTP_PROXY/HTTPS_PROXY

# Ajustar instalador (concurrencia)
poetry config installer.max-workers 8

# Dónde se guardan las configs
poetry config --list
poetry config --local --list
```

## 15) Inspección y manejo del entorno virtual de Poetry

In [ ]:
# Ruta del venv de este proyecto
# poetry env info

# Listar entornos manejados por Poetry
# poetry env list

# Eliminar un entorno (si quieres recrearlo limpio)
# poetry env remove <python|path>

# Forzar reinstalación limpia (útil tras cambiar versión mayor de Python)
# rm -rf .venv
# poetry install

## 16) Troubleshooting común (pyenv + Poetry)

- **Me mezcla versiones de Python**: ejecuta `pyenv local <versión>`, `hash -r`, y confirma con `python -V`. Asegura `virtualenvs.prefer-active-python = true`.
- **Poetry crea el venv fuera del proyecto**: activa `virtualenvs.in-project true`.
- **SSL/compilación falla**: asegúrate de tener dev‑libs (`libssl-dev`, `libffi-dev`, `libsqlite3-dev`, etc.) antes de `pyenv install`.
- **No toma la versión pineada**: verifica que estás en el root del repo (donde está `.python-version`), y que `python` de la shell corresponde (`which python`).
- **Quiero `requirements.txt` para AWS Lambda/containers**: usa `poetry export -f requirements.txt -o requirements.txt --without-hashes`.
- **Permisos con pipx/Poetry**: ejecuta `pipx ensurepath` y recarga la shell (`exec $SHELL -l`).
- **Conflictos de dependencias**: revisa `poetry.lock`, intenta `poetry update` dirigido y considera fijar rangos más precisos en `pyproject.toml`.

## 17) Notas para Windows/WSL

- Usa WSL para compilar Pythons con pyenv sin fricción.
- En Windows nativo, considera el instalador oficial de Python y Poetry (vía `pipx`) y evita rutas con espacios.
- Si trabajas cruzado (Windows ↔ WSL), mantén los entornos y paths coherentes por lado; **no** mezcles venvs entre filesystems distintos.

## 18) Migración rápida

**Desde pip/venv “a pelo”:**
1. Crea `pyproject.toml` con `poetry init`.
2. `poetry add ...` para tus dependencias.
3. `poetry install` para crear lock + venv.
4. Reemplaza `pip install -r requirements.txt` por `poetry install` (y exporta si hace falta).

**Desde uv:**
- Reemplaza `uv python pin` por `pyenv local`.
- Reemplaza `uv add/sync` por `poetry add` / `poetry install`.
- Scripts: `uv run ...` → `poetry run ...`.
- Si tenías `uv.lock`, rehace el lock con `poetry lock`.

## 19) `pyproject.toml` de ejemplo (grupos y scripts)

Ejemplo listo para copiar y adaptar.

```toml
[tool.poetry]
name = "mi-proyecto"
version = "0.1.0"
description = "Ejemplo Poetry"
authors = ["Tu Nombre <tu@email>"]
readme = "README.md"
packages = [{ include = "mi_proyecto", from = "src" }]

[tool.poetry.dependencies]
python = ">=3.12,<3.13"
boto3 = "^1.35.0"
pandas = "^2.2.0"

[tool.poetry.group.dev.dependencies]
pytest = "^8.0.0"
ruff = "^0.6.9"
black = "^24.10.0"

[tool.poetry.scripts]
app = "mi_proyecto.cli:main"
tests = "pytest -q"

[build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"
```

---

### ✅ Resumen de flujo recomendado (pyenv + Poetry)
```bash
# 1) pyenv: instalar y fijar versión por proyecto
pyenv install 3.12.4
cd ~/proyectos/mi-repo && pyenv local 3.12.4 && hash -r && python -V

# 2) Poetry: crear pyproject, venv y deps
poetry config virtualenvs.in-project true
poetry config virtualenvs.prefer-active-python true
poetry init -n
poetry add boto3 pandas
poetry install

# 3) Ejecutar y mantener
poetry run python -V
poetry run python src/main.py
poetry add --group dev pytest ruff
poetry export -f requirements.txt -o requirements.txt --without-hashes
```

¿Quieres que lo adapte a tus repos (`lla-len-mul`, lambdas, etc.) con ejemplos de `pyproject.toml` y scripts reales de tu estructura? Puedo generarlo.